1.Web Crawling from Indeed.com.
Extracting data for Job Title, Location and Summary of the Job and storing it in CSV called PMScraping.csv
Note: Could not find Salary for any of the filtered jobs, hence have not considered this field.

In [ ]:
#Installing Selenium
!pip install selenium

In [ ]:
#Importing all the necessary libraries required for Data Collection
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
import os
import csv
import pandas as pd
from math import ceil
from pandas import Series, DataFrame
import sys
from bs4 import BeautifulSoup
import requests
import json
import urllib.request


In [ ]:
#Location of the chrome driver is stored in "Chrome_driver" which is required while launching the chrome browser
Chrome_driver = "C:/Users/ditsaxen/Downloads/chromedriver_win32/chromedriver.exe"

In [ ]:
#Invoking the webdriver
driver = webdriver.Chrome(Chrome_driver)

In [ ]:
#Maximizing the chrome window
driver.maximize_window()

In [ ]:
#Here, three empty list are declared, which will store the data, belonging to the fields which we are going to extract
JobTitle = []
JobLocation = []
SummaryOfJob= []

In [ ]:
#Initialize a for loop which will iterate through 10 pages. This is used for pagination
for i in range(0,100,10):
    
        #Storing the URL of the page, this is cooked up URL , which will help us in pagination  
        URL_Pagination = "https://www.indeed.com/jobs?q=product+manager&l=San+Francisco%2C+CA" + "&start=" + str(i)
        
        #Printing the URL after each page navigation
        print(URL_Pagination)
        
        #Opening the URl to start extracting the data
        driver.get(URL_Pagination)
        
        #Storing the Job title for all the jobs dispalyed on the page
        Job_title = driver.find_elements_by_xpath("//div[@class='title']//a[text()]")
        JobTitle.extend([y.text for y in Job_title])
        
        #Storing the Job Location for all the jobs dispalyed on the page
        Job_location=driver.find_elements_by_xpath("//span[@class='location accessible-contrast-color-location']")
        JobLocation.extend([z.text for z in Job_location])
        
        #Storing the Job Summary for all the jobs dispalyed on the page
        Summary_of_the_Job=driver.find_elements_by_xpath("//div[@class='jobsearch-SerpJobCard unifiedRow row result clickcard']//div[@class='summary']")
        SummaryOfJob.extend([v.text for v in Summary_of_the_Job])

In [ ]:
#Declaring a dictionary to store all the three list
dict={'JobTitle':JobTitle,'JobLocation':JobLocation,'JobSummary':SummaryOfJob}

In [ ]:
#Converting a dictionary into Dataframe , inorder to convert it into a csv
df=pd.DataFrame(dict)

In [ ]:
#View first few entries of the data frame
df.head(20)

In [ ]:
#Finally, dataframe is converted to a csv called PMscraping.csv
df.to_csv('PMscraping.csv',index=False) 

2.Web Crawling from Naukri.com
In this Problem, we are trying to identify the Key tech hires by different organizations in Hyderabad. We will collect data for Organization Name, Skill Sets related to job posted and No of Positions in each Organization and store the final result in CSV called "Hyderabad.csv" 
By Extracting data for Hyderabad, we can get an estimate on people in Hyderabad are hiring for which skill set the most, which company is hiring the most and who have more number of open positions.
NOTE: Extracted the code for 2 pages since data for 1 month is available and one of the for loop in my code is taking more amount of time while iterating through each page. 
The Locator values on Naukri page changes in case of any updates or after few days, hence Xpath might change, which could result in locator not found.


In [ ]:
#Here, we are setting the path of chrome driver in "Chrome_driver" variable,in order to launch the Website using Chrome Browser.
Chrome_driver = "C:/Users/ditsaxen/Downloads/chromedriver_win32/chromedriver.exe"

In [ ]:
#Webdriver interface is used to launch the chrome browser. Path of the chrome driver is passed here using the variable "Chrome_driver" 
driver = webdriver.Chrome(Chrome_driver)

In [ ]:
#Since, by default the chrome browser does not open in maximize mode, so we will use maximize method to 
driver.maximize_window()

In [ ]:
#Storing the URL of Naukri.com in "url" variable.
url="https://www.naukri.com/browse-jobs"
#using get functoon to launch the URL. Once we declare driver.get(url), Naukri.com will open in the chrome browser in maximized window.
driver.get(url)

In [ ]:
#Applying filter in Search Jobs input field. Applied "Information Technology" filter. Providing xpath of the Search box to locate it.
SearchJobs=driver.find_element_by_xpath("//input[@placeholder='Skills, Designations, Companies']")

In [ ]:
#After locating the Search box , doing send keys to send the text "Informaton Technology" in the Search box input field.
SearchJobs.send_keys("Information Technology")

In [ ]:
#Locating the Location Input field to filter Hyderabad Jobs.
Location=driver.find_element_by_xpath("//input[@placeholder='Location/Locality']")
#Doing send keys to send the location "Hyderabad" in the input field.
Location.send_keys("Hyderabad")

In [ ]:
#After applying the filter on "Search Jobs" and "Location", we will locate submit button to search hyderabad specific jobs.
Search_Button=driver.find_element_by_xpath("//button[@id='qsbFormBtn']")


In [ ]:
#Clicking on Submit Button to extract result from Hyderabad region jobs.
Search_Button.click()

In [ ]:
#We have declared four empty lists here which we will use going forward in the code.
Job_Organization=[]
Job_Skills=[]
Job_Positions=[]
Job_links=[]


In [ ]:
#Initiating a for loop for pagination , to fetch content from multiple pages. From each page, content for Organization name,
#Skill set corresponding to each jobpost, and links for all the job posted on each page will be stored in three list
for i in range(1,3,1):
        URL_Pagination = "https://www.naukri.com/information-technology-jobs-in-hyderabad-secunderabad" + "-page-" + str(i)
        #Print each URL
        print (URL_Pagination)
        #Fetching URl
        driver.get(URL_Pagination)
        #storing the individual job container of each page in the div_data
        div_data = driver.find_elements_by_xpath('//div[@class="container fl"]//div[@type="tuple"]')
        
        #storing organisation name and skills in the list: Job_Organization and Job_Skills
        cnt = 0
        for job in div_data:
            #applied try catch block, it will check if element exist or not and store data accordingly. Store 'NA' if not found
            try:
                Job_Org = job.find_elements_by_xpath("//span[@class='orgRating']/span[@class='org']")[cnt].text
            except:
                Job_Org='NA'
            Job_Organization.append(Job_Org)
            try:
                Skills = job.find_elements_by_xpath("//div[@class='more desc']/span[@class='skill']")[cnt].text
            except:
                Skills='NA'
            Job_Skills.append(Skills)
            cnt+=1
            
        #Storing links for all the jobposts                 
        Pos_links=driver.find_elements_by_xpath("//a[@id='jdUrl']")
        for links in Pos_links:
            print(links.get_attribute('href'))
            Job_links.extend([links.get_attribute('href')])


        

In [ ]:
#Declared a empty list "Open", its going to store the no of positions.
Open=[]

In [ ]:
#This for loop will fetch individual link from the list Job_links, will get the data for no of openings and store it in Open List
for j in Job_links:
    driver.get(j)
    Positions=driver.find_element_by_xpath("//span[contains(text(),'Openings:')]/strong")
    o=Positions.text
    print(o)
    Open.append(o)
    


In [ ]:
#Declaring a list called " Opening" to store and convert the content of Open from string to int
Opening = []
for i in Open:
    Opening.append(int(i))


In [ ]:
#Defined a dicitionary containing data for Organization name, Skills, and No of positions.
dict={'Organization Name':Job_Organization,'Skill Set':Job_Skills,'No of Positions':Opening}



In [ ]:
#Creating a data frame from the dictionary  
Dataset=pd.DataFrame(dict)


In [ ]:
#Using head function to view first few entries of the dataset
Dataset.head(60)

In [ ]:
#Using group by on No of positions to fetech no of positions overall in each company. Storing the final content in Final_dataset
Final_Dataset = Dataset.groupby(['Organization Name','Skill Set']).sum()

In [ ]:
#displaying the content of FInal_dataset
Final_Dataset

In [ ]:
#Storing the refined data in CSV
Final_Dataset.to_csv("Hyderabad.csv")

3.Scrapping from API
Extracting data from the API "NewsApi.org"
In this problem, we are extracting the data for Source ID, Source Name, Author, Title, Description and Content and storing the final result in CSV called "news_api.csv" 
After extracting this data we can go through all the articles posted by different author, and get a glimpse of content description and title. We also get an information on source ID and Source name for each article.

In [ ]:
#storing the URL in url variable
url='https://newsapi.org/v2/top-headlines?'


In [ ]:
#Decalring the parameters, to be used while fetching data from the URL. This will apply filter of US, Business and API key.
qryes = {'country':'us',
         'category':'business',
         'apiKey': 'f980e98a08b749368923a125d83bf3c8'}

In [ ]:
#storing the final url in response variable and displaying it
response = requests.get(url, params = qryes)
response.url

In [ ]:
#Storing the response in json format in r
r=response.json()


In [ ]:
#Displaying content stored in r
r

In [ ]:
#Declaring empty lists, to store data for each field going forward.
source_id=[]
source_name=[]
author=[]
title=[]
description=[]
content=[]

In [ ]:
#Initiating a for loop to iterate through multiple pages to fetch data, and store in the empty lists declared above
for y in range(0,4,1):
    cookedurl=requests.get(url+'page='+str(y),  params = qryes)
    print(cookedurl.url)
    r1 = cookedurl.json()
    
    #A for loop to fetch content from each page for each of the desired fields and store in the empty lists
    for x in range(0,20,1):
        a=r1['articles'][x]['source']['id']
        b=r1['articles'][x]['source']['name']
        c=r1['articles'][x]['author']
        d=r1['articles'][x]['title']
        e=r1['articles'][x]['description']
        f=r1['articles'][x]['content']
    #for y in response['articles']:
        source_id.append(a)
        source_name.append(b)
        author.append(c)
        title.append(d)
        description.append(e)
        content.append(f)
    
    

In [ ]:
#once all the lists are created, we are declaring a dictionary to store the lists.
dict={'Source-ID':source_id,'Source-Name':source_name,'Author':author,'Title':title,'Description':description,'Content':content}
#dict={'Source-Name':source_name,'Author':author,'Title':title,'Description':description,'Content':content}

In [ ]:
#Displaying total no of entries available in our API
print(len(source_id))
print(len(source_name))
print(len(author))
print(len(title))
print(len(description))
print(len(content))

In [ ]:
#Storing the dictionary's content in the data frame
df=pd.DataFrame(dict)

In [ ]:
#Display total no of entries in the dataframe
len(df)

In [ ]:
#dropping duplicate entries, if any
df.drop_duplicates()

In [ ]:
#Storing final content of data frame in the CSV
df.to_csv('News_Api.csv',index=False)